<a href="https://colab.research.google.com/github/tubagokhan/ADGM/blob/main/ParsingIntermediateFormat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json

def process_file(input_file_path, output_file_path):
    data = []
    current_entry = None
    inside_table = False  # Variable to track if we're inside a table

    with open(input_file_path, 'r') as file:
        for line in file:
            if '/Table Start' in line:
                inside_table = True  # Start of table
                current_entry['Text'] += '\n' + line.strip()  # Append the line marking the start of the table
                continue  # Skip to the next line
            elif '/Table End' in line:
                inside_table = False  # End of table
                current_entry['Text'] += '\n' + line.strip()  # Append the line marking the end of the table
                continue  # Skip to the next line

            if inside_table:
                # If we're inside a table, just append the line to the current text
                current_entry['Text'] += '\n' + line.strip()
            elif line.startswith('PART ') or line.startswith('APP ') or line[0].isdigit():
                # Split the line at the first tab to separate ContextID and Text
                parts = line.split('\t', 1)
                context_id = parts[0].strip()  # ContextID is the first part
                text = parts[1].strip() if len(parts) > 1 else ''  # Text is the second part, if it exists

                # If there's an existing entry, save it before starting a new one
                if current_entry:
                    data.append(current_entry)

                # Start a new entry
                current_entry = {'ContextID': context_id, 'Text': text}
            elif current_entry:
                # Append the current line to the existing 'Text', with a newline
                current_entry['Text'] += '\n' + line.strip()

    # Make sure the last entry is added
    if current_entry:
        data.append(current_entry)

    # Save the data to a JSON file
    with open(output_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Replace 'input.txt' and 'output.json' with your actual file paths
process_file('/content/FUNDS_VER08.040723.txt', '/content/FUNDS_VER08.040723.json')
